CLASSIFICATION

In [19]:
import sktime
import sktime.transformations.panel.dictionary_based as dict_based

print(dir(dict_based))


['PAAlegacy', 'SAXlegacy', 'SFA', 'SFAFast', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_paa', '_sax', '_sfa', '_sfa_fast']


In [22]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import sys

from sktime.transformations.panel.dictionary_based import PAAlegacy

from sktime.distances import distance
from tslearn.metrics import dtw as tsldtw, dtw_path, cdist_dtw, subsequence_cost_matrix
from sklearn.metrics import pairwise_distances

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import LeaveOneOut

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold

from dtaidistance import dtw
import random

from sklearn.neighbors import sort_graph_by_row_values
import scipy.sparse as sp



In [23]:
def fast_dtw_25(ts1, ts2):
    return dtw.distance_fast(ts1, ts2, window=25)
def load_npy(filename):
    return np.load(filename)
X_y_ids = load_npy('X_y_ids.npz') 
X, y, ids = X_y_ids['arr_0'], X_y_ids['arr_1'], X_y_ids['arr_2']
df = pd.DataFrame(np.squeeze(X))
df["genre"] = y
df["ids"] = ids

X_ts = df.iloc[:,:-2].values
y_ts = df["genre"].values

X_ts.shape, y_ts.shape

((10000, 1280), (10000,))

In [24]:
%%time

paa = PAAlegacy(num_intervals=256)

def approximate(ts):
    ts_paa = paa.fit_transform(ts)
    return ts_paa

transformed_time_series = []
for ts in X_ts:
    transformed_time_series.append(approximate(ts))
    
X_ts_256 = np.array(transformed_time_series)
X_ts_256 = X_ts_256.reshape(10000, 256)

CPU times: total: 37.5 s
Wall time: 41.9 s


In [25]:
X_ts_256.shape, y_ts.shape

((10000, 256), (10000,))

In [26]:
%%time
DisMat_10k_256_DTW_fast_25 = pairwise_distances(X_ts_256, metric=fast_dtw_25)

KeyboardInterrupt: 